#**Telecom** X - Análisis de Evasión de **Clientes**

##Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os


#📌 Extracción

In [2]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

response = requests.get(url)
data = response.json()

In [3]:
df = pd.DataFrame(data)
df.dtypes

,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


In [4]:
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

In [5]:
print(df.columns.tolist())

['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


In [6]:
nestedData_col = ['customer', 'phone', 'internet', 'account']

for col in nestedData_col:
    nestedData = pd.json_normalize(df[col])
    nestedData.columns = [f"{col}_{subcol}".lower() for subcol in nestedData.columns]
    df = pd.concat([df.drop(columns=[col]), nestedData], axis=1)


df.columns = df.columns.str.replace(' ', '_').str.lower()

In [7]:
df.head()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [8]:
ruta_backup = '../content/backup/telecomx_datos.csv'


if not os.path.exists('../content/backup/'):
    os.makedirs('../content/backup/')
    print("Carpeta '../content/backup/' creada.")
else:
    print("Carpeta '../content/backup/' ya existe.")


df_backup = df.copy()
print("Backup en memoria creado.")


df_backup.to_csv(ruta_backup, index=False)
print(f"Backup guardado correctamente en '{ruta_backup}'.")

Carpeta '../content/backup/' creada.
Backup en memoria creado.
Backup guardado correctamente en '../content/backup/telecomx_datos.csv'.


In [9]:
df_data = pd.read_csv('../content/backup/telecomx_datos.csv')

In [10]:
filas = df_data.shape[0]
columnas = df_data.shape[1]

print(f"Total de filas: {filas}, \nTotal de columnas: {columnas}")

Total de filas: 7267, 
Total de columnas: 21


In [11]:
df_data.columns

Index(['customerid', 'churn', 'customer_gender', 'customer_seniorcitizen',
       'customer_partner', 'customer_dependents', 'customer_tenure',
       'phone_phoneservice', 'phone_multiplelines', 'internet_internetservice',
       'internet_onlinesecurity', 'internet_onlinebackup',
       'internet_deviceprotection', 'internet_techsupport',
       'internet_streamingtv', 'internet_streamingmovies', 'account_contract',
       'account_paperlessbilling', 'account_paymentmethod',
       'account_charges.monthly', 'account_charges.total'],
      dtype='object')

In [12]:
df_limpiar_datos = df_data.copy()

In [13]:
if 'churn' in df_limpiar_datos.columns:
    df_limpiar_datos['churn'] = df_limpiar_datos['churn'].map({'Yes': 'Canceló', 'No': 'Retenido'})
    print("Cambios realizados.")
else:
    print("Atención: Columna 'churn' no encontrada.")

Cambios realizados.


In [14]:
if 'customer_seniorcitizen' in df_limpiar_datos.columns:
    df_limpiar_datos['customer_seniorcitizen'] = df_limpiar_datos['customer_seniorcitizen'].map({0: 'No', 1: 'Sí'})
    print("Cambios realizados.")
else:
    print("Atención: Columna 'customer_seniorcitizen' no encontrada.")

Cambios realizados.


In [15]:
booleanas = [
    'account_paperlessbilling', 'customer_dependents', 'customer_partner',
    'internet_onlinesecurity', 'internet_onlinebackup', 'internet_deviceprotection',
    'internet_techsupport', 'internet_streamingtv', 'internet_streamingmovies',
    'phone_phoneservice', 'phone_multiplelines'
]

procesadas = []
no_encontradas = []

for col in booleanas:
    if col in df_limpiar_datos.columns:
        df_limpiar_datos[col] = df_limpiar_datos[col].replace({'Yes': 'Sí', 'No': 'No'})
        procesadas.append(col)
    else:
        no_encontradas.append(col)

print(f"Columnas procesadas: {procesadas}")
if no_encontradas:
    print(f"Columnas no encontradas: {no_encontradas}")

Columnas procesadas: ['account_paperlessbilling', 'customer_dependents', 'customer_partner', 'internet_onlinesecurity', 'internet_onlinebackup', 'internet_deviceprotection', 'internet_techsupport', 'internet_streamingtv', 'internet_streamingmovies', 'phone_phoneservice', 'phone_multiplelines']


In [16]:
df_limpiar_datos.head()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total
0,0002-ORFBO,Retenido,Female,No,Sí,Sí,9,Sí,No,DSL,...,Sí,No,Sí,Sí,No,One year,Sí,Mailed check,65.6,593.3
1,0003-MKNFE,Retenido,Male,No,No,No,9,Sí,Sí,DSL,...,No,No,No,No,Sí,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Canceló,Male,No,No,No,4,Sí,No,Fiber optic,...,No,Sí,No,No,No,Month-to-month,Sí,Electronic check,73.9,280.85
3,0011-IGKFF,Canceló,Male,Sí,Sí,No,13,Sí,No,Fiber optic,...,Sí,Sí,No,Sí,Sí,Month-to-month,Sí,Electronic check,98.0,1237.85
4,0013-EXCHZ,Canceló,Female,Sí,Sí,No,3,Sí,No,Fiber optic,...,No,No,Sí,Sí,No,Month-to-month,Sí,Mailed check,83.9,267.4


#📊 Carga y análisis

In [18]:
hay_nulos = False

print("Columnas con valores nulos y su cantidad:")

for columna in df_limpiar_datos.columns:
    nulos = df_limpiar_datos[columna].isna().sum()
    if nulos > 0:
        print(f"{columna}: {nulos}")
        hay_nulos = True

if not hay_nulos:
    print("No se encontraron valores nulos en ninguna columna.")

Columnas con valores nulos y su cantidad:
churn: 224


In [19]:
df_eliminar_nulos = df_limpiar_datos.copy()

In [20]:
df_eliminar_nulos.head()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total
0,0002-ORFBO,Retenido,Female,No,Sí,Sí,9,Sí,No,DSL,...,Sí,No,Sí,Sí,No,One year,Sí,Mailed check,65.6,593.3
1,0003-MKNFE,Retenido,Male,No,No,No,9,Sí,Sí,DSL,...,No,No,No,No,Sí,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Canceló,Male,No,No,No,4,Sí,No,Fiber optic,...,No,Sí,No,No,No,Month-to-month,Sí,Electronic check,73.9,280.85
3,0011-IGKFF,Canceló,Male,Sí,Sí,No,13,Sí,No,Fiber optic,...,Sí,Sí,No,Sí,Sí,Month-to-month,Sí,Electronic check,98.0,1237.85
4,0013-EXCHZ,Canceló,Female,Sí,Sí,No,3,Sí,No,Fiber optic,...,No,No,Sí,Sí,No,Month-to-month,Sí,Mailed check,83.9,267.4


In [21]:
df_eliminar_nulos = df_eliminar_nulos.query(' & '.join([f'`{col}` == `{col}`' for col in df.columns]))

In [22]:
hay_nulos = False

print("Columnas con valores nulos y su cantidad:")

for columna in df_eliminar_nulos.columns:
    nulos = df_eliminar_nulos[columna].isna().sum()
    if nulos > 0:
        print(f"{columna}: {nulos}")
        hay_nulos = True

if not hay_nulos:
    print("No se encontraron valores nulos en ninguna columna.")

Columnas con valores nulos y su cantidad:
No se encontraron valores nulos en ninguna columna.


In [23]:
datos_conservados = round(100 * len(df_eliminar_nulos) / len(df_limpiar_datos), 2)
registros_eliminados = len(df_limpiar_datos) - len(df_eliminar_nulos)

print(f"Registros eliminados: {registros_eliminados}")
print(f"Porcentaje de datos conservados: {datos_conservados}%")

Registros eliminados: 224
Porcentaje de datos conservados: 96.92%


In [24]:
df_eliminar_nulos.head()

,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total
0,0002-ORFBO,Retenido,Female,No,Sí,Sí,9,Sí,No,DSL,...,Sí,No,Sí,Sí,No,One year,Sí,Mailed check,65.6,593.3
1,0003-MKNFE,Retenido,Male,No,No,No,9,Sí,Sí,DSL,...,No,No,No,No,Sí,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Canceló,Male,No,No,No,4,Sí,No,Fiber optic,...,No,Sí,No,No,No,Month-to-month,Sí,Electronic check,73.9,280.85
3,0011-IGKFF,Canceló,Male,Sí,Sí,No,13,Sí,No,Fiber optic,...,Sí,Sí,No,Sí,Sí,Month-to-month,Sí,Electronic check,98.0,1237.85
4,0013-EXCHZ,Canceló,Female,Sí,Sí,No,3,Sí,No,Fiber optic,...,No,No,Sí,Sí,No,Month-to-month,Sí,Mailed check,83.9,267.4


In [25]:
df_eliminar_nulos.describe()

,customer_tenure,account_charges.monthly
count,7043.000000,7043.000000
mean,32.371149,64.761692
std,24.559481,30.090047
min,0.000000,18.250000
25%,9.000000,35.500000
50%,29.000000,70.350000
75%,55.000000,89.850000
max,72.000000,118.750000


In [26]:
df_eliminar_nulos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7043 non-null   object 
 1   churn                      7043 non-null   object 
 2   customer_gender            7043 non-null   object 
 3   customer_seniorcitizen     7043 non-null   object 
 4   customer_partner           7043 non-null   object 
 5   customer_dependents        7043 non-null   object 
 6   customer_tenure            7043 non-null   int64  
 7   phone_phoneservice         7043 non-null   object 
 8   phone_multiplelines        7043 non-null   object 
 9   internet_internetservice   7043 non-null   object 
 10  internet_onlinesecurity    7043 non-null   object 
 11  internet_onlinebackup      7043 non-null   object 
 12  internet_deviceprotection  7043 non-null   object 
 13  internet_techsupport       7043 non-null   object 
 1

In [27]:
valores_unicos = df_eliminar_nulos.nunique().reset_index()
valores_unicos.columns = ['Columna', 'Valores Únicos']
valores_unicos = valores_unicos.sort_values(by='Valores Únicos')
print(valores_unicos)

                      Columna  Valores Únicos
1                       churn               2
2             customer_gender               2
3      customer_seniorcitizen               2
4            customer_partner               2
7          phone_phoneservice               2
5         customer_dependents               2
17   account_paperlessbilling               2
9    internet_internetservice               3
12  internet_deviceprotection               3
13       internet_techsupport               3
10    internet_onlinesecurity               3
11      internet_onlinebackup               3
14       internet_streamingtv               3
15   internet_streamingmovies               3
16           account_contract               3
8         phone_multiplelines               3
18      account_paymentmethod               4
6             customer_tenure              73
19    account_charges.monthly            1585
20      account_charges.total            6531
0                  customerid     

In [28]:
df_eliminar_nulos.isnull().sum().sort_values(ascending=False)

,0
customerid,0
churn,0
customer_gender,0
customer_seniorcitizen,0
customer_partner,0
customer_dependents,0
customer_tenure,0
phone_phoneservice,0
phone_multiplelines,0
internet_internetservice,0


#📄Informe final